In [4]:
import os
import torch
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 模型选择需与训练时保持一致
model_name = 'mobilenet_v3'  # 请确保与 train_model.py 中的 model_name 一致

def create_model(model_name=model_name):
    if model_name.lower() == 'resnet50':
        model = models.resnet50(pretrained=False)
        in_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    elif model_name.lower() == 'mobilenet_v3':
        model = models.mobilenet_v3_large(pretrained=False)
        in_features = model.classifier[0].in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    elif model_name.lower() == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=False)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    else:
        raise ValueError("model_name 仅支持 'resnet50'、'mobilenet_v3' 或 'efficientnet_b0'")
    return model.to(device)

# 图像预处理
img_height = 150
img_width = 150
test_transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor()
])

# 测试数据路径
test_dir = r'C:\Users\wzf20\Desktop\chest_xray_pneumonia\data\test'
# 使用 ImageFolder 加载测试数据，要求测试集文件夹按照类别组织
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 加载模型
model = create_model(model_name)
model_path = r'C:\Users\wzf20\Desktop\chest_xray_pneumonia\models\model.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

all_preds = []
# 从 test_dataset.samples 中提取相对路径（ImageFolder 会自动分配标签）
image_paths = [os.path.relpath(path, test_dir) for (path, _) in test_dataset.samples]

with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = (torch.sigmoid(outputs) >= 0.5)
        preds = preds.cpu().numpy().astype(int).flatten()
        all_preds.extend(preds.tolist())

# 生成提交文件
submission_df = pd.DataFrame({
    'image_id': image_paths,
    'predictions': all_preds
})

submission_path = r'C:\Users\wzf20\Desktop\chest_xray_pneumonia\results\predictions\submission.csv'
os.makedirs(os.path.dirname(submission_path), exist_ok=True)
submission_df.to_csv(submission_path, index=False)
print(f"提交文件已保存到 {submission_path}")

C:\Users\wzf20\AppData\Local\Temp\ipykernel_6036\780092045.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

提交文件已保存到 C:\Users\wzf20\Desktop\chest_xray_pneumonia\results\predictions\submission.csv
